# Dataset 2

moving to another notebook bc I'm getting impatient

In [18]:
import pandas as pd
import numpy as np
import regex as re

#notify me when a long running cell is complete



SyntaxError: invalid syntax (3122122601.py, line 6)

In [3]:
# Reading 2nd dataset file
dataset2 = pd.read_csv('data/combined_data_2.txt',header = None, names = ['cust_id', 'rating', 'date'])
dataset2.head()

,cust_id,rating,date
0,4500:,NaN,NaN
1,2532865,4.0,2005-07-26
2,573364,3.0,2005-06-20
3,1696725,3.0,2004-02-27
4,1253431,3.0,2004-03-31


In [4]:
dataset2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26982302 entries, 0 to 26982301
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   cust_id  object 
 1   rating   float64
 2   date     object 
dtypes: float64(1), object(2)
memory usage: 617.6+ MB


In [5]:
# only movie id vals

mid_index2 = pd.DataFrame(dataset2[(dataset2['rating'].isnull())])
mid_index2.head()

,cust_id,rating,date
0,4500:,NaN,NaN
259,4501:,NaN,NaN
855,4502:,NaN,NaN
960,4503:,NaN,NaN
2740,4504:,NaN,NaN


In [6]:
#rename movie index
mid_index2 = mid_index2.rename(columns = {'cust_id': 'movie_id'})
mid_index2.head()

,movie_id,rating,date
0,4500:,NaN,NaN
259,4501:,NaN,NaN
855,4502:,NaN,NaN
960,4503:,NaN,NaN
2740,4504:,NaN,NaN


In [2]:
#drop movieid punctuation
def drop_punctuation(df):
    midnew = []
    for index, row in df.iterrows():
        mid1 = str(row['movie_id'])
        regex = r"[^\w\s]"
        mid1 = re.sub(regex, '', mid1)
        midnew.append(mid1)
        
    df['mid'] = midnew

    return df

In [13]:
#test = mid_index2['movie_id'][0]
#test

'4500:'

In [15]:
#test
#regex = r"[^\w\s]"
#test = re.sub(regex, '', test)
#test

'4500'

In [21]:
#drop punctuation, drop other cols
drop_punctuation(mid_index2)
mid_index2 = mid_index2[['mid']]
mid_index2.head()

,mid
0,4500
259,4501
855,4502
960,4503
2740,4504


In [23]:
mid_index2.shape

(4711, 1)

In [24]:
dataset2.head()

,cust_id,rating,date
0,4500:,NaN,NaN
1,2532865,4.0,2005-07-26
2,573364,3.0,2005-06-20
3,1696725,3.0,2004-02-27
4,1253431,3.0,2004-03-31


In [26]:
#concatenate the full dataset with midindex rows
d2 = pd.concat([dataset2, mid_index2], axis = 1)
d2.head()

,cust_id,rating,date,mid
0,4500:,NaN,NaN,4500
1,2532865,4.0,2005-07-26,NaN
2,573364,3.0,2005-06-20,NaN
3,1696725,3.0,2004-02-27,NaN
4,1253431,3.0,2004-03-31,NaN


In [27]:
#checking for mid 4504
d2.iloc[2740]

cust_id    4504:
rating       NaN
date         NaN
mid         4504
Name: 2740, dtype: object

In [28]:
d2 = d2.reindex(['mid', 'cust_id', 'rating', 'date'], axis= 1)
d2.head()

,mid,cust_id,rating,date
0,4500,4500:,NaN,NaN
1,NaN,2532865,4.0,2005-07-26
2,NaN,573364,3.0,2005-06-20
3,NaN,1696725,3.0,2004-02-27
4,NaN,1253431,3.0,2004-03-31


In [3]:
def fill_mids(df, column_name):
    """
    iterates through df col
    replaces nan vals with movie id until it reaches a new movie id
    """
    # Find the first non-NaN value in the column
    first_value = df[column_name].dropna().iloc[0]
    
    # Iterate through the column, filling NaNs with the previous non-NaN value
    for i in range(len(df)):
        if pd.isna(df.loc[i, column_name]):
            df.loc[i, column_name] = first_value
        else:
            first_value = df.loc[i, column_name]
            if pd.isna(first_value):
                first_value = df[column_name].dropna().iloc[0]

In [32]:
fill_mids(d2, 'mid')
d2.head()

,mid,cust_id,rating,date
0,4500,4500:,NaN,NaN
1,4500,2532865,4.0,2005-07-26
2,4500,573364,3.0,2005-06-20
3,4500,1696725,3.0,2004-02-27
4,4500,1253431,3.0,2004-03-31


In [34]:
d2.iloc[960]

mid         4503
cust_id    4503:
rating       NaN
date         NaN
Name: 960, dtype: object

In [36]:
#removing cust id with movie ids in them

d2['cust_id'].str.contains(':').sum()

4711

In [38]:
d2 = d2[~d2['cust_id'].str.contains(':')]
d2.shape

(26977591, 4)

In [39]:
d2['date'] = d2['date'].astype('datetime64[ns]')
d2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26977591 entries, 1 to 26982301
Data columns (total 4 columns):
 #   Column   Dtype         
---  ------   -----         
 0   mid      object        
 1   cust_id  object        
 2   rating   float64       
 3   date     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 1.0+ GB


/var/folders/xc/4fn08svj2dgdk_3wj3xkp_rr0000gn/T/ipykernel_29698/3053115047.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d2['date'] = d2['date'].astype('datetime64[ns]')


In [40]:
d2['year'] = d2['date'].dt.year
d2.head()

/var/folders/xc/4fn08svj2dgdk_3wj3xkp_rr0000gn/T/ipykernel_29698/2478966670.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d2['year'] = d2['date'].dt.year


,mid,cust_id,rating,date,year
1,4500,2532865,4.0,2005-07-26,2005
2,4500,573364,3.0,2005-06-20,2005
3,4500,1696725,3.0,2004-02-27,2004
4,4500,1253431,3.0,2004-03-31,2004
5,4500,1265574,2.0,2003-09-01,2003


In [41]:
d2['month'] = d2['date'].dt.month
d2.head()

/var/folders/xc/4fn08svj2dgdk_3wj3xkp_rr0000gn/T/ipykernel_29698/2999920986.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d2['month'] = d2['date'].dt.month


,mid,cust_id,rating,date,year,month
1,4500,2532865,4.0,2005-07-26,2005,7
2,4500,573364,3.0,2005-06-20,2005,6
3,4500,1696725,3.0,2004-02-27,2004,2
4,4500,1253431,3.0,2004-03-31,2004,3
5,4500,1265574,2.0,2003-09-01,2003,9


In [42]:
d2['day'] = d2['date'].dt.day
d2.head()

/var/folders/xc/4fn08svj2dgdk_3wj3xkp_rr0000gn/T/ipykernel_29698/340210831.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d2['day'] = d2['date'].dt.day


,mid,cust_id,rating,date,year,month,day
1,4500,2532865,4.0,2005-07-26,2005,7,26
2,4500,573364,3.0,2005-06-20,2005,6,20
3,4500,1696725,3.0,2004-02-27,2004,2,27
4,4500,1253431,3.0,2004-03-31,2004,3,31
5,4500,1265574,2.0,2003-09-01,2003,9,1


In [4]:
#quarterly bins

def quarter(row):
    if row['month'] < 4:
        return 1
    elif 4 <= row['month'] < 7:
        return 2
    elif 7 <= row['month'] < 9:
        return 3
    else:
        return 4

In [44]:
d2['fisc_Q'] = d2.apply(quarter, axis=1)
d2.head()

,mid,cust_id,rating,date,year,month,day,fisc_Q
1,4500,2532865,4.0,2005-07-26,2005,7,26,3
2,4500,573364,3.0,2005-06-20,2005,6,20,2
3,4500,1696725,3.0,2004-02-27,2004,2,27,1
4,4500,1253431,3.0,2004-03-31,2004,3,31,1
5,4500,1265574,2.0,2003-09-01,2003,9,1,4


In [ ]:
d2.to_csv('data/d2_clean.csv')

# Dataset 3

In [5]:
# Reading dataset file
dataset3 = pd.read_csv('data/combined_data_3.txt',header = None, names = ['cust_id', 'rating', 'date'])
dataset3.head()

,cust_id,rating,date
0,9211:,NaN,NaN
1,1277134,1.0,2003-12-02
2,2435457,2.0,2005-06-01
3,2338545,3.0,2001-02-17
4,2218269,1.0,2002-12-27


In [7]:
#only movie id values
mid_index3 = pd.DataFrame(dataset3[(dataset3['rating'].isnull())])
mid_index3.head()

,cust_id,rating,date
0,9211:,NaN,NaN
505,9212:,NaN,NaN
1347,9213:,NaN,NaN
1491,9214:,NaN,NaN
1856,9215:,NaN,NaN


In [8]:
#rename movie index
mid_index3 = mid_index3.rename(columns = {'cust_id': 'movie_id'})
mid_index3.head()

,movie_id,rating,date
0,9211:,NaN,NaN
505,9212:,NaN,NaN
1347,9213:,NaN,NaN
1491,9214:,NaN,NaN
1856,9215:,NaN,NaN


In [9]:
#drop punctuation fxn defined above
drop_punctuation(mid_index3)
mid_index3 = mid_index3[['mid']]
mid_index3.head()

,mid
0,9211
505,9212
1347,9213
1491,9214
1856,9215


In [10]:
mid_index3.shape

(4157, 1)

In [11]:
dataset3.head()

,cust_id,rating,date
0,9211:,NaN,NaN
1,1277134,1.0,2003-12-02
2,2435457,2.0,2005-06-01
3,2338545,3.0,2001-02-17
4,2218269,1.0,2002-12-27


In [15]:
d3 = pd.concat([dataset3, mid_index3], axis = 1 )
d3 = d3.reindex(['mid', 'cust_id', 'rating', 'date'], axis= 1)
d3.head()

,mid,cust_id,rating,date
0,9211,9211:,NaN,NaN
1,NaN,1277134,1.0,2003-12-02
2,NaN,2435457,2.0,2005-06-01
3,NaN,2338545,3.0,2001-02-17
4,NaN,2218269,1.0,2002-12-27


In [16]:
fill_mids(d3, 'mid')
d3.head()

,mid,cust_id,rating,date
0,9211,9211:,NaN,NaN
1,9211,1277134,1.0,2003-12-02
2,9211,2435457,2.0,2005-06-01
3,9211,2338545,3.0,2001-02-17
4,9211,2218269,1.0,2002-12-27


In [17]:
d3.iloc[1856]

mid         9215
cust_id    9215:
rating       NaN
date         NaN
Name: 1856, dtype: object

In [19]:
#removing cust id with movie ids in them

d3['cust_id'].str.contains(':').sum()

4157

In [20]:
d3 = d3[~d3['cust_id'].str.contains(':')]
d3.shape

(22601629, 4)

In [25]:
d3.iloc[504]

mid              9212
cust_id       1378111
rating            5.0
date       2005-07-12
Name: 506, dtype: object

In [26]:
d3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22601629 entries, 1 to 22605785
Data columns (total 4 columns):
 #   Column   Dtype  
---  ------   -----  
 0   mid      object 
 1   cust_id  object 
 2   rating   float64
 3   date     object 
dtypes: float64(1), object(3)
memory usage: 862.2+ MB


In [27]:
d3['date'] = d3['date'].astype('datetime64[ns]')
d3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22601629 entries, 1 to 22605785
Data columns (total 4 columns):
 #   Column   Dtype         
---  ------   -----         
 0   mid      object        
 1   cust_id  object        
 2   rating   float64       
 3   date     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 862.2+ MB


In [28]:
d3['year'] = d3['date'].dt.year
d3.head()

,mid,cust_id,rating,date,year
1,9211,1277134,1.0,2003-12-02,2003
2,9211,2435457,2.0,2005-06-01,2005
3,9211,2338545,3.0,2001-02-17,2001
4,9211,2218269,1.0,2002-12-27,2002
5,9211,441153,4.0,2002-10-11,2002


In [29]:
d3['month'] = d3['date'].dt.month
d3.head()

,mid,cust_id,rating,date,year,month
1,9211,1277134,1.0,2003-12-02,2003,12
2,9211,2435457,2.0,2005-06-01,2005,6
3,9211,2338545,3.0,2001-02-17,2001,2
4,9211,2218269,1.0,2002-12-27,2002,12
5,9211,441153,4.0,2002-10-11,2002,10


In [30]:
d3['day'] = d3['date'].dt.day
d3.head()

,mid,cust_id,rating,date,year,month,day
1,9211,1277134,1.0,2003-12-02,2003,12,2
2,9211,2435457,2.0,2005-06-01,2005,6,1
3,9211,2338545,3.0,2001-02-17,2001,2,17
4,9211,2218269,1.0,2002-12-27,2002,12,27
5,9211,441153,4.0,2002-10-11,2002,10,11


In [31]:
d3['fisc_Q'] = d3.apply(quarter, axis=1)
d3.head()

,mid,cust_id,rating,date,year,month,day,fisc_Q
1,9211,1277134,1.0,2003-12-02,2003,12,2,4
2,9211,2435457,2.0,2005-06-01,2005,6,1,2
3,9211,2338545,3.0,2001-02-17,2001,2,17,1
4,9211,2218269,1.0,2002-12-27,2002,12,27,4
5,9211,441153,4.0,2002-10-11,2002,10,11,4


In [ ]:
d3.to_csv('data/d3_clean.csv')